<a href="https://colab.research.google.com/github/aaubs/ds-master/blob/main/notebooks/M3_agent_1_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agents 🤖

Agents are like "tools" for LLMs. They allow an LLM to access Google search, perform complex calculations with Python, and even make SQL queries.

In this notebook, we'll explore agents and how to use them in LangChain.

We'll start by installing the prerequisite libraries that we'll be using in this example.

# Built-in Agents in LangChain

| **Agent Type**             | **Description**                                                                                                                                                   | **How to Build**                                                                                                                                                      | **Tools Used**                     |
|-----------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------|-------------------------------------|
| **Zero Shot React**         | An agent that uses an LLM to determine actions without prior context or memory. Suitable for tasks that don't require conversation history or complex reasoning.  | Use `create_sql_agent` with `SQLDatabaseToolkit`, specifying `agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION`.                                                     | `SQLDatabaseToolkit`               |
| **Conversational React**    | An agent capable of maintaining conversation history and reasoning over previous interactions. Ideal for interactive applications that require memory.            | Use `initialize_agent` with `AgentType.CONVERSATIONAL_REACT_DESCRIPTION`, include a memory like `ConversationBufferMemory`, and load necessary tools (e.g., llm-math). | `llm-math` tool, `ConversationBufferMemory` |
| **React Docstore**          | An agent that interacts with a docstore to search and retrieve information. Useful for answering questions based on documents or databases.                      | Use `initialize_agent` with `AgentType.REACT_DOCSTORE`, and include tools like `WikipediaAPIWrapper` for document retrieval.                                         | `WikipediaAPIWrapper`              |
| **Self Ask with Search**    | An agent that decomposes questions into sub-questions and uses a search engine to find intermediate answers, leading to a final answer. Best for queries requiring external information. | Use `initialize_agent` with `AgentType.SELF_ASK_WITH_SEARCH`, and include search tools like `SerpAPIWrapper`. Note: Requires an API key for the search service.       | `SerpAPIWrapper` (Search Tool)     |
| **Plan and Execute**        | An agent that creates a plan of actions to achieve a goal before executing them sequentially. Useful for complex tasks requiring multi-step reasoning.            | Use `initialize_agent` with `AgentType.PLAN_AND_EXECUTE`, and include necessary tools for executing each step of the plan.                                           | Various execution tools             |
| **Agent Executor**          | A generic agent that can execute a series of tools based on the input prompt and internal logic. Versatile for multiple use cases.                              | Use `initialize_agent` with `AgentType.EXECUTOR`, and configure with desired tools and logic.                                                                          | Various tools based on use case     |
| **Agent Synthesis**         | An agent that synthesizes information from multiple sources to provide comprehensive answers. Ideal for aggregating data from various APIs or databases.         | Use `initialize_agent` with `AgentType.SYNTHESIS`, and include tools that access different data sources.                                                                | Multiple data access tools          |
| **Dynamic Agent**           | An agent that dynamically selects tools and actions based on the context and requirements of the task.                                                        | Use `initialize_agent` with `AgentType.DYNAMIC`, and set up conditional logic for tool selection.                                                                       | Conditional tools logic             |
| **Tool Use Agent**          | An agent designed to utilize a predefined set of tools to perform specific tasks. Ideal for automating workflows that require tool interactions.                 | Use `initialize_agent` with `AgentType.TOOL_USE`, and specify the tools to be integrated.                                                                              | Predefined set of tools             |
| **ReAct**                   | An agent that combines reasoning and acting, allowing it to perform tasks that require both understanding and executing actions based on that understanding.     | Use `initialize_agent` with `AgentType.REACT`, and integrate tools that support reasoning and action execution.                                                        | Reasoning and action execution tools|
| **Agent Zero-Shot**         | Similar to Zero Shot React but optimized for scenarios requiring minimal configuration and quick deployment without extensive customization.                   | Use `create_agent` with `AgentType.ZERO_SHOT`, focusing on default settings for rapid implementation.                                                                    | Minimal configuration tools         |

---

**Notes:**

- **Tool Integration:** Each agent type utilizes specific tools tailored to its functionality. Ensure that the necessary tools are installed and properly configured in your environment.
  
- **API Keys:** Some agents, especially those involving search functionalities like `Self Ask with Search`, may require API keys. Make sure to obtain and securely store any required credentials.
  
- **Customization:** While these agents come built-in, LangChain allows for extensive customization. You can extend or modify agents to better fit your specific use cases.

- **Documentation:** For the most up-to-date and detailed information, always refer to the [official LangChain documentation](https://langchain.readthedocs.io/).



In [ ]:
!pip install -qU langchain openai sqlalchemy wikipedia langchain_community --q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.5/389.5 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.3 MB/s eta 0:00:00


To run this notebook, we will need to use an OpenAI LLM. Here we will set up the LLM we will use for the whole notebook. Just input your OpenAI API key when prompted.

In [ ]:
from getpass import getpass

OPENAI_API_KEY = getpass("Enter your OpenAI API key:")

Enter your OpenAI API key:··········


Now, we can initialize the LLM.

In [ ]:
from langchain.llms import OpenAI

llm = OpenAI(
    openai_api_key=OPENAI_API_KEY,
    temperature=0
)

<ipython-input-5-9e0b12c5cbe2>:3: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(


As we did before, we will be counting our tokens in each call.

In [ ]:
from langchain.callbacks import get_openai_callback

def count_tokens(agent, query):
    with get_openai_callback() as cb:
        result = agent(query)
        print(f'Spent a total of {cb.total_tokens} tokens')
    return result

With all of that set up, let's jump into Agents.

## What is an agent?

**Definition:** The key behind agents is giving LLMs the possibility of using tools in their workflow. This is where LangChain departs from the popular ChatGPT implementation and we can start to get a glimpse of what it offers us as builders. Until now, we covered several building blocks in isolation. Let's see them come to life.

The official definition of agents is the following:

**Agents use an LLM to determine which actions to take and in what order. An action can either be using a tool and observing its output or returning to the user.**

In this notebook, we will cover several types of agents and see how they 'think' and what they can do.

## Create a Database

We will use the agents to interact with a small sample database of movies. We will not dive into the details because this is just a dummy tool we will build for illustrative purposes. Let's create it.

First, we create the database schema.

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Sample Movies Data
movies_data = [
    {'id': 1, 'title': 'The Shawshank Redemption', 'director': 'Frank Darabont', 'year': 1994, 'genre': 'Drama', 'rating': 9.3},
    {'id': 2, 'title': 'The Godfather', 'director': 'Francis Ford Coppola', 'year': 1972, 'genre': 'Crime', 'rating': 9.2},
    {'id': 3, 'title': 'The Dark Knight', 'director': 'Christopher Nolan', 'year': 2008, 'genre': 'Action', 'rating': 9.0},
    {'id': 4, 'title': 'Pulp Fiction', 'director': 'Quentin Tarantino', 'year': 1994, 'genre': 'Crime', 'rating': 8.9},
    {'id': 5, 'title': 'The Lord of the Rings: The Return of the King', 'director': 'Peter Jackson', 'year': 2003, 'genre': 'Adventure', 'rating': 8.9},
    {'id': 6, 'title': "Schindler's List", 'director': 'Steven Spielberg', 'year': 1993, 'genre': 'Biography', 'rating': 8.9},
    {'id': 7, 'title': 'Forrest Gump', 'director': 'Robert Zemeckis', 'year': 1994, 'genre': 'Drama', 'rating': 8.8},
    {'id': 8, 'title': 'Inception', 'director': 'Christopher Nolan', 'year': 2010, 'genre': 'Action', 'rating': 8.8},
    {'id': 9, 'title': 'Fight Club', 'director': 'David Fincher', 'year': 1999, 'genre': 'Drama', 'rating': 8.8},
    {'id': 10, 'title': 'The Matrix', 'director': 'Lana Wachowski, Lilly Wachowski', 'year': 1999, 'genre': 'Sci-Fi', 'rating': 8.7}
]

# Create DataFrame
df_movies = pd.DataFrame(movies_data)

# Database Configuration
DATABASE_URL = "sqlite:///movies.db"  # SQLite database file named 'movies.db'

# Create SQLAlchemy Engine
engine = create_engine(DATABASE_URL, echo=True)  # echo=True enables SQL logging

# Insert DataFrame into 'movies' Table
df_movies.to_sql('movies', con=engine, if_exists='replace', index=False)

# Read Data Back into a DataFrame
df_result = pd.read_sql('SELECT * FROM movies', con=engine)

# Display the DataFrame
df_result

## Agent Types

In this section, we will review several agents and see how they 'think' and what they can do.

Using one of LangChain's pre-built agents involves three variables:

1. Defining the tools or the toolkit
2. Defining the LLM
3. Defining the agent type

### Agent Type #1: Zero Shot React

In this first example, we will use the Zero Shot React agent to interact with our movies database.

We will use the `create_sql_agent` function. This method uses a toolkit instead of a simple list of tools.

Now we are ready to initialize the agent! We will use `verbose=True` so we can see what our agent's 'thinking' process.

**Important Note:** When interacting with agents, it is really important to set the `max_iterations` parameter because agents can get stuck in infinite loops that consume plenty of tokens. The default value is 15 to allow for many tools and complex reasoning, but for most applications, you should keep it much lower.

In [ ]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType
from langchain.utilities import SQLDatabase

db = SQLDatabase(engine)
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    max_iterations=2
)


In [ ]:
tables_query = "SELECT name FROM sqlite_master WHERE type='table';"  # Use SQLite syntax to get table names
result = db.run(tables_query)
print(result)

[('movies',)]


In [ ]:
records_query = "SELECT * FROM movies LIMIT 10;"  # Replace 'table_name' with the actual table name
records_result = db.run(records_query)
print(records_result)

Let's see our newly created agent in action! We will ask it a question that involves an aggregation over the movie ratings.

In [ ]:
result = count_tokens(
    agent_executor,
    "Which director has the highest average movie rating?"
)




> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: moviesI should query the schema of the movies table to see what columns I can use.
Action: sql_db_schema
Action Input: movies2024-11-16 09:10:36,675 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2024-11-16 09:10:36,680 INFO sqlalchemy.engine.Engine SELECT movies.id, movies.title, movies.director, movies.year, movies.genre, movies.rating 
FROM movies
 LIMIT ? OFFSET ?


INFO:sqlalchemy.engine.Engine:SELECT movies.id, movies.title, movies.director, movies.year, movies.genre, movies.rating 
FROM movies
 LIMIT ? OFFSET ?


2024-11-16 09:10:36,684 INFO sqlalchemy.engine.Engine [generated in 0.00902s] (3, 0)


...

2024-11-16 09:10:37,953 INFO sqlalchemy.engine.Engine [generated in 0.00521s] ()


INFO:sqlalchemy.engine.Engine:[generated in 0.00521s] ()


2024-11-16 09:10:37,957 INFO sqlalchemy.engine.Engine COMMIT


INFO:sqlalchemy.engine.Engine:COMMIT


[('Frank Darabont', 9.3), ('Francis Ford Coppola', 9.2), ('Steven Spielberg', 8.9), ('Quentin Tarantino', 8.9), ('Peter Jackson', 8.9), ('Christopher Nolan', 8.9), ('Robert Zemeckis', 8.8), ('David Fincher', 8.8), ('Lana Wachowski, Lilly Wachowski', 8.7)] I now know the final answer.
Final Answer: Frank Darabont

> Finished chain.
Spent a total of 0 tokens


**Sample Output:**

```vbnet
> Entering new AgentExecutor chain...
Thought: I need to query the database to find the average rating per director and then find the director with the highest average rating.
Action: query_sql_db
Action Input: SELECT director, AVG(rating) as avg_rating FROM movies GROUP BY director ORDER BY avg_rating DESC LIMIT 1;

Observation: [('Frank Darabont', 9.3)]
Thought: I have the result.
Final Answer: Frank Darabont has the highest average movie rating.

> Finished chain.
Spent a total of 150 tokens
```


### Agent Type #2: Conversational React

The Zero Shot agent is interesting but has no memory. What if we want an assistant that remembers things we have talked about and can also reason about them and use tools? For that, we have the Conversational React agent.

We will use the math tool in this example and load it as below:

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.memory import ConversationBufferMemory
from langchain.agents.agent_types import AgentType

tools = load_tools(["llm-math"], llm=llm)
memory = ConversationBufferMemory(memory_key="chat_history")

conversational_agent = initialize_agent(
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=2,
    memory=memory
)

<ipython-input-33-f172c9865266>:7: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history")
<ipython-input-33-f172c9865266>:9: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  conversational_agent = initialize_agent(


Now, let's have a conversation with our agent.

First, we ask a math question.

In [ ]:
result = count_tokens(
    conversational_agent,
    "If I have 3 movies rated 8.5, 9.0, and 7.5, what is their average rating?"
)




> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: Average of 8.5, 9.0, and 7.5
Observation: Answer: 8.333333333333334
Thought: Do I need to use a tool? No
AI: The average rating of the three movies is 8.333333333333334.

> Finished chain.
Spent a total of 1160 tokens


**Sample Output:**

```vbnet
> Entering new AgentExecutor chain...
Thought: I need to calculate the average of the three ratings.
Action: Calculator
Action Input: (8.5 + 9.0 + 7.5) / 3
Observation: Answer: 8.333333333333334
Thought: I now have the average rating.
AI: The average rating of the three movies is approximately 8.33.

> Finished chain.
Spent a total of 132 tokens
```


Then, let's ask a follow-up question.

In [ ]:
result = count_tokens(
    conversational_agent,
    "If I watch another movie rated 9.5, what will be the new average rating?"
)




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: (8.5 + 9.0 + 7.5 + 9.5) / 4
Observation: Answer: 8.625
Thought: Do I need to use a tool? No
AI: The new average rating will be 8.625.

> Finished chain.
Spent a total of 1261 tokens


**Sample Output:**

```vbnet
> Entering new AgentExecutor chain...
Thought: I should add the new rating and recalculate the average.
Action: Calculator
Action Input: (8.5 + 9.0 + 7.5 + 9.5) / 4
Observation: Answer: 8.625
Thought: I have the new average rating.
AI: The new average rating after watching the fourth movie is 8.625.

> Finished chain.
Spent a total of 142 tokens
```


This demonstrates the agent's ability to remember previous interactions and use tools to perform calculations.

### Agent Type #3: React Docstore

This type of agent is similar to the ones we have seen so far but it includes the interaction with a docstore. It will have tools at its disposal to search and retrieve information.

Let's use the Wikipedia tool to answer questions.

In [ ]:
from langchain import Wikipedia
from langchain.agents.react.base import DocstoreExplorer, Tool

docstore = DocstoreExplorer(Wikipedia())

tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description='search wikipedia'
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description='lookup a term in wikipedia'
    )
]

Now, let's ask a question.

In [ ]:
docstore_agent = initialize_agent(
    tools,
    llm,
    agent="react-docstore",
    verbose=True,
    max_iterations=2
)

In [ ]:
result = count_tokens(
    docstore_agent,
    "Who directed 'Inception' and what other popular movies has this director made?"
)




> Entering new AgentExecutor chain...
Thought: I need to search Inception, find its director, then find other popular movies made by the director.
Action: Search[Inception]
Observation: Could not find [Inception]. Similar: ['Inception', 'Inception (disambiguation)', 'Inception (soundtrack)', 'Peugeot Inception', 'Fréchet inception distance', 'Inception (deep learning architecture)', 'Into Inception', 'Ultimate tic-tac-toe', 'Nvidia', 'List of accolades received by Inception']
Thought: I need to search Inception (film) instead.
Action: Search[Inception (film)]
Observation: Inception is a 2010 science fiction  action  heist  film written and directed by Christopher Nolan, who also produced it with Emma Thomas, his wife. The film stars Leonardo DiCaprio as a professional thief who steals information by infiltrating the subconscious of his targets. He is offered a chance to have his criminal history erased as payment for the implantation of another person's idea into a target's subconsci

**Sample Output:**

```vbnet
> Entering new AgentExecutor chain...
Thought: I need to find out who directed 'Inception' and then find other movies by the same director.
Action: Wikipedia
Action Input: Inception
Observation: "Inception" is a 2010 science fiction action film written and directed by Christopher Nolan...
Thought: The director is Christopher Nolan. Now I need to find other popular movies by him.
Action: Wikipedia
Action Input: Christopher Nolan filmography
Observation: Christopher Nolan has directed films like 'Memento', 'The Dark Knight', 'Interstellar', and 'Dunkirk'.
Thought: I have the information now.
Final Answer: 'Inception' was directed by Christopher Nolan. Other popular movies he has directed include 'Memento', 'The Dark Knight', 'Interstellar', and 'Dunkirk'.

> Finished chain.
Spent a total of 352 tokens
```


This shows how the agent uses Wikipedia to find information about Christopher Nolan and mentions other popular movies he has directed.

### Agent Type #4: Self Ask with Search

This is the first-choice agent to use when using LLMs to extract information with a search engine. The agent will ask follow-up questions and use the search functionality to get intermediate answers that help it get to a final answer.

Considering the image below, it is evident that Self-Ask Prompting is a progression from Direct and Chain-Of-Thought prompting.

The interesting thing about self-ask prompting is that the LLM reasoning is shown explicitly and the LLM also decomposes the question into smaller follow-up questions.

The LLM knows when the final answer is reached and can move from follow up intermediate answers to a final answer.

![](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*5R8lx3gTlEjvvHoSs0Gjhg.png)

We will use the `SerpAPIWrapper` to perform searches. Note that you need a SerpAPI key to use this tool.

In [ ]:
!pip install google-search-results openai langchain --q

  Preparing metadata (setup.py) ... done


In [ ]:
from getpass import getpass

OPENAI_API_KEY = getpass("Enter your OpenAI API key:")

Enter your OpenAI API key:··········


In [ ]:
SERPAPI_API_KEY = getpass("Enter your SerpAPI API key:")

Enter your SerpAPI API key:··········


In [ ]:
import os

os.environ["SERPAPI_API_KEY"] = SERPAPI_API_KEY

from langchain import OpenAI, SerpAPIWrapper
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

llm = OpenAI(
    openai_api_key=OPENAI_API_KEY,
    temperature=0
)
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="useful for when you need to ask with search",
    )
]

self_ask_with_search = initialize_agent(
    tools, llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True
)
self_ask_with_search.run(
    "What is the hometown of the reigning men's U.S. Open champion?"
)

<ipython-input-11-914424018aed>:22: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  self_ask_with_search = initialize_agent(
<ipython-input-11-914424018aed>:25: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  self_ask_with_search.run(




> Entering new AgentExecutor chain...
 Yes.
Follow up: Who is the reigning men's U.S. Open champion?
Intermediate answer: Jannik Sinner
Follow up: Where is Jannik Sinner from?
Intermediate answer: Jannik Sinner (born August 16, 2001, San Candido, Italy) is an Italian tennis player who is a leading star in the generation of players rising in the wake of the indomitable Big Three (Novak Djokovic, Roger Federer, and Rafael Nadal).
So the final answer is: San Candido, Italy

> Finished chain.


'San Candido, Italy'

In [ ]:
print(self_ask_with_search.agent.llm_chain.prompt.template)

Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washin

![](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*cWt_8xW7Ajz-BJ0TSZ63Iw.png)

## Wrapping Up

And that's all for agents! There's many other things you can do with agents, just to name a few:

- Create your own custom agent
- Use them with many other tools (even custom ones)
- Trace every call an agent makes through a convenient UI interface
- Check the how-to guides for more!

In this modified notebook, we have replaced the previous stock database with a movies database, which is more engaging and interesting. We have also provided more tangible examples for each agent type, demonstrating their capabilities in a clear and understandable way.

Feel free to experiment further and explore the vast possibilities that agents offer in LangChain!